<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/bert_replace/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
!pip install keras_bert

In [2]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

fatal: destination path 'bachelor_design' already exists and is not an empty directory.
/content/bachelor_design


In [3]:
!git checkout bert_replace
!git pull

Already on 'bert_replace'
Your branch is up to date with 'origin/bert_replace'.
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/supplient/bachelor_design
   ca2ea43..c3694a9  bert_replace -> origin/bert_replace
Updating ca2ea43..c3694a9
Fast-forward
 preprocess.py | 13 +++++++------
 1 file changed, 7 insertions(+), 6 deletions(-)


# Mount GDriver
Perpare Colab environment.

In [2]:
from driver_amount import addh

# Prepare Input

In [1]:
import preprocess
import config

Using TensorFlow backend.
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tenso

In [4]:
char_seqs, tag_seqs = preprocess.load_file(
    addh + config.DATA_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN
    )

In [5]:
import pickle
with open(addh + config.TAG_VOCAB_PATH, "wb") as fd:
    pickle.dump(tag_vocab, fd)

In [11]:
tag_vocab

{'': 0,
 'B-EI': 6,
 'B-EIF': 10,
 'B-EO': 4,
 'B-EQ': 8,
 'B-ILF': 2,
 'I-EI': 7,
 'I-EIF': 11,
 'I-EO': 5,
 'I-EQ': 9,
 'I-ILF': 3,
 'O': 1}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [13]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # checkpoint file
    addh + config.BERT_CHECKPOINT_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

output = keras.layers.Dense(len(tag_vocab), activation=keras.activations.softmax)(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [15]:
model.compute_output_shape([(1, config.SEQ_LEN), (1, config.SEQ_LEN)])

(1, 512, 12)

In [16]:
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [17]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 512, 768)     177262848   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512, 12)      9228        model_2[1][0]              

# Fine-tuning

In [18]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"]
)

In [8]:
from util import EpochCheckpoint
epoch_callback = EpochCheckpoint(config.MODEL_PATH, config.TRAIN_LOG_PATH, 1)

AttributeError: module 'config' has no attribute 'TRAIN_LOG_PATH'

In [0]:
model.fit(
    x=[token_id_seqs, segment_seqs],
    y=[tag_id_seqs], 
    batch_size=8, 
    epochs=5,
    verbose=2,
    validation_split=0.1,
    callbacks=[epoch_callback]
)

Train on 4999 samples, validate on 2785 samples
Epoch 1/5
 - 868s - loss: 0.5371 - acc: 0.8336 - val_loss: 0.8042 - val_acc: 0.6658
Epoch 2/5
 - 876s - loss: 0.3678 - acc: 0.8738 - val_loss: 0.7756 - val_acc: 0.7009
Epoch 3/5
 - 879s - loss: 0.2975 - acc: 0.8952 - val_loss: 0.7109 - val_acc: 0.7613
Epoch 4/5
 - 878s - loss: 0.2455 - acc: 0.9127 - val_loss: 0.8333 - val_acc: 0.7190
Epoch 5/5
 - 877s - loss: 0.2038 - acc: 0.9277 - val_loss: 0.8154 - val_acc: 0.7191


In [0]:
model.save(addh + config.MODEL_PATH)